In [1]:
import glob
import os
import paramiko
import pandas as pd
import numpy as np
import datetime

print(os.getcwd())
print(datetime.datetime.now())

/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q3
2020-08-04 10:43:54.827467


In [2]:
quarter="2020Q3"
writer_upload_files_folder="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_%s/upload_files/"%quarter
try:
    os.stat(writer_upload_files_folder)
except:
    os.mkdir(writer_upload_files_folder)
list_all=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_%s/wanted_segment_files/*"%quarter)
len(list_all)

40

In [3]:
list_all_C=[x for x in list_all if os.path.basename(x)[0]=="C"]

print(len(list_all_C))

20


In [4]:
pd.read_csv(list_all_C[0],nrows=10)

,customer_id_hashed,email_address_hash,Store_Type,HML_Group,Status,EmailSub,segment_2020Q3
0,bbb02589dc717718c03480d9424d8450f33a6e158072e5...,f11f2329b2b066b9a039931c0f1963ed8ff221c352be72...,Legacy,H,Active,N,C_Legacy_H_Active_N
1,7c058afa0a6e15371f0b27c39228ff3b5bc25dd1c46174...,d2c663a798bfb99e05b0845c6285390f6e04e20de2ac81...,Legacy,H,Active,N,C_Legacy_H_Active_N
2,270071128fc233e866c1990f0528d6f8583afee65675c7...,2eaf012beaf55622f3378708e2badcdc7ee5e73f72d8c3...,Legacy,H,Active,N,C_Legacy_H_Active_N
3,669c72b64c5068f1a9c3b64a6faf9b2052025264ec4b0d...,b662fdeae7d8558587123e915573234da967ffde456210...,Legacy,H,Active,N,C_Legacy_H_Active_N
4,4d6fb5ab42bfe61a00618a6ce811a2d1214a32f78c94f4...,58869f80a66136f4531136259dbb9ac6ddc0925d7fa27c...,Legacy,H,Active,N,C_Legacy_H_Active_N
5,5abc69fd6047c618367245164b8596b3aaf47ceef003f6...,9663d00f0d0126c9ec62daa344165fa7730fa0118a63e3...,Legacy,H,Active,N,C_Legacy_H_Active_N
6,1ea15eef4a77d5627e416f6419ee96a2cd3f3c10ddf410...,da4a0aba28f345553725a7517fcea4064c8a5c3e518fbe...,Legacy,H,Active,N,C_Legacy_H_Active_N
7,7c8ff9aed48bf562fec04e39b1934ccc7a4234ab6f5823...,e429706c833beae9c4145156119bcc450d15c8ef39cfa5...,Legacy,H,Active,N,C_Legacy_H_Active_N
8,d073e952c77cbb93571325ab05deedb922b17a6e7bc2a3...,dafb7a5b423fc2b0a8e151c10613050ea696a600d4300a...,Legacy,H,Active,N,C_Legacy_H_Active_N
9,632d75b0c2dacc4c0939c9895d9537f3806e336d2269d3...,77df3ee6d6714f5370bf0f39b5ba22c85e550b0fe6db4e...,Legacy,H,Active,N,C_Legacy_H_Active_N


In [5]:
df_aggregated_control=pd.DataFrame()

for file in list_all_C:
    df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','email_address_hash','segment_%s'%quarter])
    df_aggregated_control=df_aggregated_control.append(df)
df_aggregated_control['segment_%s'%quarter]="C_aggregated_for_2020Q3"
df_aggregated_control.to_csv(writer_upload_files_folder+"C_unified_quarterly_segments_RFM%s.csv"%quarter[-4:],index=False)
print(datetime.datetime.now(),df_aggregated_control.shape)

2020-08-04 10:45:55.246631 (599982, 3)


In [6]:
# Upload to LR

In [7]:
list_all_files_to_upload=glob.glob(writer_upload_files_folder+"*")
list_all_files_to_upload=[x for x in list_all_files_to_upload if "unified" in x]
list_all_files_to_upload

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q3/upload_files/C_unified_quarterly_segments_RFM20Q3.csv']

In [10]:
import paramiko
host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)


In [11]:
list_all_files_to_upload

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q3/upload_files/C_unified_quarterly_segments_RFM20Q3.csv']

In [12]:
for local_file in list_all_files_to_upload:
    base_name=os.path.basename(local_file)
    remote_name="/uploads/big_lots_onboarding/"+base_name
    sftp.put(local_file,remote_name)
    print(datetime.datetime.now(),remote_name)
print("All_Done: "+str(datetime.datetime.now()))
sftp.close()
transport.close()

2020-08-04 11:06:24.572094 /uploads/big_lots_onboarding/C_unified_quarterly_segments_RFM20Q3.csv
All_Done: 2020-08-04 11:06:24.573972
